<a href="https://colab.research.google.com/github/Bara7-Qusai/hugginface/blob/main/LoRA_Image_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install diffusers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
import requests
from io import BytesIO

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
# تحميل النموذج مع المصادقة (قد يتطلب token من Hugging Face)
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_auth_token=True  # أضف التوكن هنا إذا لزم الأمر
).to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
def load_image(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)).convert("RGB")
    except Exception as e:
        print(f"خطأ في تحميل الصورة: {e}")
        return None


In [5]:
image_url = "https://cdn.salla.sa/ZorEx/GMTGWqD4cDQfyXKtXT3CMcQdldFR5U5IlHIUvLzd.jpg"  # استبدل بالرابط الفعلي
input_image = load_image(image_url)

In [21]:
if input_image:
    # معالجة مسبقة للصورة
    input_image = input_image.resize((512, 512))  # تغيير الحجم للحفاظ على تناسق النموذج

    # إعداد الإعدادات
    prompt = "A high-quality, professional marketing photo of the product with stunning, photorealistic details. Keep the product unchanged, maintaining its original appearance and textures. Replace the background with a cosmic-inspired design featuring vibrant, well-matched colors that enhance the product’s appeal. Use soft nebula clouds, glowing stars, or smooth galaxy hues to create an elegant and mesmerizing cosmic atmosphere. Ensure the background blends harmoniously with the product, making it stand out beautifully while maintaining a clean, premium aesthetic. Studio-quality rendering with perfect lighting and realistic depth."
    num_images = 3

    # توليد الصور
    results = pipe(
        prompt=prompt,
        image=input_image,
        strength=0.6,  # تقليل القيمة للحفاظ على تفاصيل المنتج
        num_inference_steps=95,
        guidance_scale=8.0,
        num_images_per_prompt=num_images  # إصلاح: توليد عدة صور
    )
    for i, img in enumerate(results.images):
        img.save(f"product_variant_{i+1+2}.jpg", quality=95)  # حفظ كصورة JPEG بجودة عالية
        print(f"تم حفظ الصورة رقم {i+1}")

    print("🎉 اكتملت العملية بنجاح!")
else:
    print("فشل في تحميل الصورة الأصلية")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['. ensure the background blends harmoniously with the product , making it stand out beautifully while maintaining a clean , premium aesthetic . studio - quality rendering with perfect lighting and realistic depth .']


  0%|          | 0/57 [00:00<?, ?it/s]

تم حفظ الصورة رقم 1
تم حفظ الصورة رقم 2
تم حفظ الصورة رقم 3
🎉 اكتملت العملية بنجاح!
